# Модель для анализ чеков

In [2]:
import pandas as pd
from parser import Parser
import os
import re
from  sklearn.feature_extraction.text import CountVectorizer

from pipline import remove_unit_whitespace,add_whitespace_before_unit_once,split_digit_with_unit,remove_service_char,remove_general_words

In [2]:
parser = Parser()

In [275]:
files = os.listdir("data")

df = pd.DataFrame()

for filename in files:
    new_df = parser.parse_to_df(filename)
    df = pd.concat(
        [df,new_df],axis=0,ignore_index=True
    )

In [695]:
df = pd.read_csv("data.csv",index_col='ID')

In [696]:
df['clr_name'] = (
    df.name
    .apply(add_whitespace_before_unit_once)
    .apply(split_digit_with_unit)
    .apply(remove_nX)
    .apply(remove_service_char)
)

In [697]:
df.head()

,name,price,sum,quantity,paymentType,productType,nds,ndsSum,date,label,clr_name
ID,,,,,,,,,,,
0,РОГАЛИК МОЛОЧ.4Х85Г,6899,6899,1.0,4,1,2,627,2021-01-09,1.0,рогалик молоч г
1,ХЛЕБЦЫ КУКУРУЗ.75Г,2199,2199,1.0,4,1,2,200,2021-01-09,1.0,хлебцы кукуруз г
2,ХЛЕБЦЫ ОГО СТ 75Р,2119,2119,1.0,4,1,2,193,2021-01-09,1.0,хлебцы ого ст р
3,РОГАЛИК МОЛОЧ.4Х85Г,6899,6899,1.0,4,1,2,627,2021-01-09,1.0,рогалик молоч г
4,МАКФА СПАГЕТТИ 500Г,5999,5999,1.0,4,1,2,545,2021-01-09,5.0,макфа спагетти г


In [698]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from  sklearn.metrics import f1_score


In [700]:

text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', SGDClassifier(loss='hinge', penalty='l2',
 alpha=1e-3, random_state=42,
 max_iter=5, tol=None)),
 ])


In [701]:
X = df[~df.label.isnull()].drop('label',axis=1)
y = df[~df.label.isnull()].label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)



In [702]:
text_clf.fit(X_train.clr_name,y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [703]:

# X_tfidf = tf_transformer.transform(X_count)
predict = text_clf.predict(X_test.clr_name)
f1_score(y_test,predict,average=None)

array([0.8       , 0.5       , 0.57142857, 0.        , 0.22222222,
       0.71428571, 0.66666667, 1.        , 0.33333333, 0.76923077])

In [759]:
df = pd.read_csv("data.csv",index_col='ID')

df['clr_name'] = (
    df.name
    .apply(add_whitespace_before_unit_once)
    .apply(split_digit_with_unit)
    .apply(remove_nX)
    .apply(remove_service_char)
    .str.replace("%"," ")
)

X = df[~df.label.isnull()].drop('label',axis=1)
y = df[~df.label.isnull()].label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# text_clf.fit(X_train.clr_name,y_train)

predict = text_clf.predict(X_test.clr_name)
f1_score(y_test,predict,average=None)



array([1.        , 0.875     , 0.96296296, 1.        , 0.93333333,
       0.875     , 0.96296296, 1.        , 1.        , 0.91428571])

In [756]:
X_test['true']= y_test
X_test['pred'] = predict
X_test[X_test.true==1]

<ipython-input-756-21ed8ab8b7b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['true']= y_test
<ipython-input-756-21ed8ab8b7b2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['pred'] = predict


,name,price,sum,quantity,paymentType,productType,nds,ndsSum,date,clr_name,true,pred
ID,,,,,,,,,,,,
183,СЛОЙКА ВЕТЧ/СЫР 1Х75,2699,2699,1.000,4,1,2,245,2021-04-30,слойка ветч/сыр,1.0,1.0
158,БАРАНКИ ВВ МАК300Г,3999,3999,1.000,4,1,2,364,2021-04-30,баранки вв мак г,1.0,1.0
790,ХЛЕБЦ ЩЕДР БОРОД 170,5499,5499,1.000,4,1,2,500,2021-01-15,хлебц щедр бород,1.0,1.0
246,СЛОЙКА ВЕТЧ/СЫР 1Х75,2699,2699,1.000,4,1,2,245,2021-03-27,слойка ветч/сыр,1.0,1.0
235,БАТОН ДАЧНЫЙ НАР.350,3999,3999,1.000,4,1,2,364,2021-03-27,батон дачный нар,1.0,1.0
82,ЛАВАШ 220Г,3599,3599,1.000,4,1,2,327,2021-01-06,лаваш г,1.0,1.0
73,ЛЕПЁШКА С СЫРОМ 290Г,4299,4299,1.000,4,1,2,391,2021-01-06,лепёшка с сыром г,1.0,1.0
182,СЛОЙКА ВЕТЧ/СЫР 1Х75,2699,2699,1.000,4,1,2,245,2021-04-30,слойка ветч/сыр,1.0,1.0
452,БАТОН ДАЧНЫЙ НАР.350,3999,3999,1.000,4,1,2,364,2021-02-06,батон дачный нар,1.0,1.0


In [762]:


X_train = df[~df.label.isnull()]
y_train = X_train.label
X_test = df[df.label.isnull()]

text_clf.fit(X_train.clr_name,y_train)
predict = text_clf.predict(X_test.clr_name)

X_test['label'] = predict

<ipython-input-762-0a8d30fadb77>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['label'] = predict


In [765]:

X = pd.concat([X_train,X_test],axis=0,ignore_index=True)

n=10
for i in range(n+1):
    X[X.label==i].to_csv(f"categories/{i}.csv")

In [768]:
df = parser.parse_to_df("07_05_2021_07_09_396895281735009403388.json")

df['clr_name'] = (
    df.name
    .apply(add_whitespace_before_unit_once)
    .apply(split_digit_with_unit)
    .apply(remove_nX)
    #pply(remove_service_char)
    # ..replace("%"," ")
)
predict = text_clf.predict(df.clr_name)
df['label'] = predict
